# Import libraries

In [6]:
%pip install mat73
%pip install nibabel
%pip install hdf5storage

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


                                              0.0/53.6 kB ? eta -:--:--
     ----------------------                   30.7/53.6 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 53.6/53.6 kB 556.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [4]:
import nibabel as nib
import torch
from torch import nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from os.path import join as opj
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

import tqdm
from torchsummary import summary
from PIL import Image

from sklearn.model_selection import train_test_split
import shutil
from scipy.stats import hypergeom
import seaborn as sns
from termcolor import colored
from sklearn.neighbors import NearestNeighbors
from torchvision.utils import make_grid
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
import nibabel.processing
import pickle
from scipy.io import loadmat
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from god_decoding_utils import *

from sklearn.metrics import classification_report

from torch import autocast
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline

from PIL import Image
import ast
import matplotlib as plt
import wandb
from PIL import ImageFilter

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MehrSystem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MehrSystem\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Data loading

In [5]:
# Initialize sub and roi variables
# You can change sub to each number in [1-5]
sub = "3"
roi = "ROI_VC"

# Initialize kamitani_data_mat with the path to a file containing data for Subject 3
kamitani_data_mat = f"data/fMRI/GOD/Subject{sub}.h5"

# Initialize imagenet_dir with the path to a directory containing images
imagenet_dir = "data/fMRI/imagenet/images"

# Initialize test_img_csv and train_img_csv with the paths to two CSV files
test_img_csv = 'data/fMRI/GOD/image_test_id.csv'
train_img_csv = 'data/fMRI/GOD/image_training_id.csv'
test_img_csv
train_img_csv
# Create a data_handler object using the specified arguments
handler = data_handler(h5_file=kamitani_data_mat, test_img_csv=test_img_csv, train_img_csv=train_img_csv)

# Get data using the get_data method of the handler object
Y, Y_test, Y_test_avg = handler.get_data(normalize=1, roi=roi)

# Get labels using the get_labels method of the handler object
labels_train, labels = handler.get_labels()

# Get filenames using the get_filenames method of the handler object
filenames_train, filenames_test = handler.get_filenames()

# Convert filenames_train and filenames_test from arrays to Python lists
filenames_train = [i.item() for i in filenames_train]
filenames_test = [i.item() for i in filenames_test]

# Initialize an empty list called filenames_test_avg
filenames_test_avg = []

# Iterate 50 times
for i in range(50):
    # Append the first element from filenames_test whose label is equal to i to filenames_test_avg
    filenames_test_avg.append(np.array(filenames_test)[labels==i][0])

# Wordnet mapping

In [6]:
with open('class_to_wordnet.json',) as f:
    data = f.read()

class2wordnet = ast.literal_eval(data)
wordnet2class= {v["id"] : k for k,v in class2wordnet.items()}

In [7]:
# Initialize img_dir_path with the path to a directory containing training images
img_dir_path = "data/fMRI/imagenet/images/training"

# Initialize image_paths with a list of file paths to the training images
image_paths = [os.path.join(img_dir_path, i) for i in filenames_train]

# Initialize test_image_paths with a list of file paths to the test images
test_image_paths = [os.path.join(imagenet_dir,"test", i) for i in filenames_test_avg]

# Split the data into train and validation sets using the train_test_split function
fmri_train, fmri_val, image_train, image_val = train_test_split(Y, image_paths, test_size=0.1, random_state=42)


# Dataset and Dataloaders

In [8]:
# Initialize BS with the batch size
BS = 64

# Initialize normalize with a transformation that normalizes the data
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Initialize transform with a transformation that resizes and converts images to tensors
transform = torchvision.transforms.Compose([torchvision.transforms.Resize(224),
                                            torchvision.transforms.ToTensor(),
                                            normalize])

# Initialize train_dataset with a CustomDataset object using the fmri_train and image_train data and the transform
train_dataset = CustomDataset(fmri_train, image_train, transform)

# Initialize val_dataset with a CustomDataset object using the fmri_val and image_val data and the transform
val_dataset = CustomDataset(fmri_val, image_val, transform)

# Initialize test_dataset with a CustomDataset object using the Y_test_avg and test_image_paths data and the transform
test_dataset = CustomDataset(Y_test_avg, test_image_paths, transform)

# Initialize train_dataloader with a DataLoader object using the train_dataset and the specified batch size
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=False)

# Initialize val_dataloader with a DataLoader object using the val_dataset and the specified batch size
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BS, shuffle=False)

# Initialize test_dataloader with a DataLoader object using the test_dataset and the specified batch size
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BS, shuffle=False)

# CNN Model

In [9]:
# Load a pre-trained ResNet50 model from the torchvision library
model = torchvision.models.resnet50(pretrained=True)
print(model)
# Get the list of child modules of the model
modules = list(model.children())

# Remove the last element of the list of child modules
modules = modules[:-1]

# Create a new Sequential model using the remaining child modules
model = nn.Sequential(*modules)

C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Extract the latent representation for each image and store it

In [10]:
# Initialize empty lists to store the data
x_train = []
z_train = []
y_train = []

x_val = []
z_val = []
y_val = []

x_test = []
z_test = []
y_test = []

# Iterate over the train_dataloader, val_dataloader, and test_dataloader
# x are fMRI preprocessed data, y are the image stimuli

with torch.no_grad():
    for x, y in tqdm.tqdm(train_dataloader):
        # Use the model to compute the latent representation of y
        z = model(y)
        # Append the data to the respective lists
        x_train.append(x)
        z_train.append(z.squeeze())
        y_train.append(y)

    for x, y in tqdm.tqdm(val_dataloader):
        # Use the model to compute the latent representation of y
        z = model(y)
        # Append the data to the respective lists
        x_val.append(x)
        z_val.append(z.squeeze())
        y_val.append(y)
    for x, y in tqdm.tqdm(test_dataloader):
        # Use the model to compute the latent representation of y
        z = model(y)
        # Append the data to the respective lists
        x_test.append(x)
        z_test.append(z.squeeze())
        y_test.append(y)

# Concatenate the lists of data into numpy arrays
x_train = torch.cat(x_train, 0).numpy()
z_train = torch.cat(z_train, 0).numpy()
y_train = torch.cat(y_train, 0).numpy()

x_val = torch.cat(x_val, 0).numpy()
z_val = torch.cat(z_val, 0).numpy()
y_val = torch.cat(y_val, 0).numpy()

x_test = torch.cat(x_test, 0).numpy()
z_test = torch.cat(z_test, 0).numpy()
y_test = torch.cat(y_test, 0).numpy()


100%|██████████| 1/1 [00:03<00:00,  3.48s/it]


In [11]:
print(z_train.shape)
print(z_test.shape)
print(z_val.shape)

(1080, 2048)
(50, 2048)
(120, 2048)


# Linear Model

In [12]:
from sklearn.model_selection import GridSearchCV

ridge=Ridge(max_iter=5000)
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,10,100,1000,10000,]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(x_train,z_train)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:255: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=7.74628e-20): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)
C:\Users\MehrSystem\AppData\Local\Programs\Python\Python310\li

{'alpha': 10000}
-0.18195555745550646


In [13]:
ridge = Ridge(alpha=10000, max_iter=5000)

ridge.fit(x_train, z_train)

Ridge(alpha=10000, max_iter=5000)

In [41]:
# Use the model to predict the latent representation of the test data
pred_test_latent = ridge.predict(x_test)

# Initialize epsilon with a small value
epsilon = 1e-10

# Standardize the predicted latent representation of the test data
std_norm_test_latent = (pred_test_latent - np.mean(pred_test_latent, axis=0)) / (epsilon + np.std(pred_test_latent, axis=0))

# Scale and shift the standardized predicted latent representation to match the scale and shift of the training data
pred_instance = std_norm_test_latent * np.std(z_train, axis=0) + np.mean(z_train, axis=0)

# Use the model to predict the latent representation of the training and validation data
train_predicted = ridge.predict(x_train)
val_predicted = ridge.predict(x_val)

# Calculate the root mean squared error (RMSE) of the predictions
print("rmse", mean_squared_error(z_train, train_predicted))
print("rmse", mean_squared_error(z_val, val_predicted))
print("rmse", mean_squared_error(z_test, pred_instance))
print("rmse", mean_squared_error(z_test, pred_test_latent))


rmse 0.03112794856925971
rmse 0.20582907075477286
rmse 0.2667757419987978
rmse 0.16935964149718472


# Perceptron

In [ ]:
fmri_dim=x_train.shape[-1]
latent_dim=z_train.shape[-1]

In [ ]:
def get_activation(activation):
    if activation=="relu":
        return nn.ReLU()

    elif activation=="gelu":
        return nn.GELU()

    elif activation=="tanh":
        return nn.Tanh()

    elif activation=="sigmoid":
        return nn.Sigmoid()

    elif activation=="selu":
        return nn.SELU()

In [ ]:
class BrainMLP(nn.Module):
    def __init__(self,latent_dim,hidden=[128],dropout=0.2,activation="gelu"):
        super().__init__()


        self.model=[]
        for h in hidden:
            self.model.append(nn.LazyLinear(h))
            self.model.append(get_activation(activation))


        self.model.append(nn.Dropout(dropout))


        self.model.append(nn.LazyLinear(latent_dim))

        self.model=nn.Sequential(*self.model)

    def forward(self,x):
        return self.model(x)

In [ ]:
# Initialize a BrainMLP model with specified latent dimension, hidden layer sizes, dropout rate, and activation function
brain_model = BrainMLP(latent_dim=latent_dim, hidden=[1024], dropout=0.3, activation="tanh")

# Print a summary of the model, including the input and output shapes and the number of parameters
summary(brain_model, (fmri_dim,), device="cpu")

In [1]:
# Convert the x_train, z_train, x_val, z_val, x_test, and z_test arrays to PyTorch tensors
train_brain_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(z_train))
val_brain_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_val), torch.Tensor(z_val))
test_brain_dataset = torch.utils.data.TensorDataset(torch.Tensor(x_test), torch.Tensor(z_test))

# Create DataLoaders for the training, validation, and test sets
train_brain_dataloader = torch.utils.data.DataLoader(train_brain_dataset, batch_size=64, shuffle=True)
val_brain_dataloader = torch.utils.data.DataLoader(val_brain_dataset, batch_size=64, shuffle=True)
test_brain_dataloader = torch.utils.data.DataLoader(test_brain_dataset, batch_size=64, shuffle=True)

NameError: name 'torch' is not defined

In [2]:
# Set the device to use for training
device = "cuda:0"

# Set the number of epochs to train for
EPOCHS = 200

# Set the step size and gamma for the learning rate scheduler
scheduler_step = 60
gamma = 0.5

# Move the brain model to the designated device
brain_model = brain_model.to(device)

# Create an Adam optimizer with a weight decay of 1e-5
optim = torch.optim.Adam(brain_model.parameters(), lr=3e-3, weight_decay=1e-5)

# Create a learning rate scheduler that reduces the learning rate by a factor of gamma every scheduler_step epochs
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=scheduler_step, gamma=gamma)

# Create a mean squared error loss function
criterion = nn.MSELoss()

# Uncomment the following line to use a cosine embedding loss instead
# criterion = nn.CosineEmbeddingLoss()
# model=model.to(device)

NameError: name 'brain_model' is not defined

In [3]:
def train_brain_epoch(model, train_dataloader, criterion=None, optim=None, device="cpu"):
    """
    Trains the model for one epoch using the specified dataloader and optimizer.

    Parameters:
    - model: PyTorch model to be trained
    - train_dataloader: Dataloader for the training data
    - criterion: Loss function to be used for training
    - optim: Optimizer to be used for training
    - device: Device to run the model and data on (CPU or GPU)

    Returns:
    - mean_loss: Mean loss of the model over the training data
    """
    model.train()

    loss_tmp = []
    # Iterate over the training data
    for x, y in train_dataloader:
        # Zero the gradients of the optimizer
        optim.zero_grad()
        # Move the data to the specified device
        x, y = x.to(device), y.to(device)
        # Get the model's prediction for the input data
        y_pred = model(x)

        # Calculate the loss based on the criterion and model's prediction
        if isinstance(criterion, nn.CosineEmbeddingLoss):
            # Set the target to 1 if using the CosineEmbeddingL
            target=torch.ones(y_pred.shape[0]).to(device)

            loss=criterion(y_pred.squeeze(),y.squeeze(),target)

        else:

            loss=criterion(y_pred.squeeze(),y.squeeze())
        # Backpropagate the loss and update the model's parameters
        loss.backward()
        optim.step()
        # Add the loss to the list of losses
        loss_tmp.append(loss.item())
    # Calculate the mean loss over all the data
    mean_loss = np.mean(loss_tmp)
    return mean_loss


In [ ]:

def val_brain_epoch(model, val_dataloader, criterion=None, optim=None, device="cpu"):
    """
    Validates the model using the specified dataloader.

    Parameters:
    - model: PyTorch model to be validated
    - val_dataloader: Dataloader for the validation data
    - criterion: Loss function to be used for validation
    - optim: Optimizer to be used for validation (not used in this function)
    - device: Device to run the model and data on (CPU or GPU)

    Returns:
    - mean_loss: Mean loss of the model over the validation data
    """
    model.eval()

    loss_tmp = []
    i = 0
    # Iterate over the validation data
    with torch.no_grad():
        for x, y in val_dataloader:
            # Move the data to the specified device
            x, y = x.to(device), y.to(device)
            # Get the model's prediction for the input data
            y_pred = model(x)
            # Calculate the loss based on the criterion and model's prediction
            if isinstance(criterion, nn.CosineEmbeddingLoss):
                # Set the target to 1 if using the CosineEmbeddingLoss criterion
                target = torch.ones(y_pred.shape[0]).to(device)
                loss = criterion(y_pred.squeeze(), y.squeeze(), target)
            else:
                loss = criterion(y_pred.squeeze(), y.squeeze())
            # Add the loss to the list of losses
            loss_tmp.append(loss.item())
    # Calculate the mean loss over all the data
    mean_loss = np.mean(loss_tmp)
    return mean_loss

In [ ]:
loss_history=[]
val_loss_history=[]

# frequency of validation
val_freq=1

train = True
# if train is set to True, train the model
if train:
    # create a progress bar
    pbar=tqdm.tqdm(range(EPOCHS))
    # iterate over the number of epochs
    for epoch in pbar:
        # call the train_brain_epoch function to train the model for one epoch
        loss=train_brain_epoch(brain_model,train_dataloader=train_brain_dataloader,criterion=criterion,optim=optim,device=device)

        # step the scheduler to adjust the learning rate
        scheduler.step()

        # append the loss to the loss history
        loss_history.append(loss)

        # if the current epoch is a multiple of val_freq, call the val_brain_epoch function to get the validation loss
        if epoch%val_freq==0:
            val_loss=val_brain_epoch(brain_model,val_dataloader=val_brain_dataloader,criterion=criterion,optim=optim,device=device)
            val_loss_history.append(val_loss)

        # update the progress bar description with the current epoch and loss information
        if epoch>val_freq:
            pbar.set_description(f"[INFO] epoch: {epoch} loss: {loss_history[-1]} val_loss: {val_loss_history[-1]}")
        else:
            pbar.set_description(f"[INFO] epoch: {epoch} loss: {loss_history[-1]}")


    #save final weights
    torch.save(brain_model.state_dict(),f"models/brain_model_weights_sub{sub}.pt")

else:
    #load pretrained models

    brain_model.load_state_dict(torch.load(f"models/brain_model_weights_sub{sub}.pt"))
    brain_model=brain_model.to(device)
    ridge = pickle.load(open(f"models/ridge_sub{sub}.sav", 'rb'))

In [ ]:
if train:
    plt.plot(loss_history,label="loss")
    plt.plot(val_loss_history,label="val_loss")
    plt.legend()

In [ ]:
# Initialize variables for predictions on train, val, and test sets
train_predict = None
val_predict = None
test_predict = None

# Turn off gradient calculation
with torch.no_grad():
  # Make predictions on train set
  train_predict = brain_model(torch.Tensor(x_train).to(device)).cpu().numpy()
  # Make predictions on val set
  val_predict = brain_model(torch.Tensor(x_val).to(device)).cpu().numpy()
  # Make predictions on test set
  test_predict = brain_model(torch.Tensor(x_test).to(device)).cpu().numpy()

  # Standardize test predictions
  std_norm_test_latent = (test_predict - np.mean(test_predict, axis=0)) / np.std(test_predict, axis=0)
  # Adjust test predictions using mean and std of training set
  test_predicted_adjusted = std_norm_test_latent * np.std(z_train, axis=0) + np.mean(z_train, axis=0)

# Calculate and print root mean squared error for train, val, and test sets
print("rmse", mean_squared_error(z_train, train_predict))
print("rmse", mean_squared_error(z_val, val_predict))
print("rmse", mean_squared_error(z_test, test_predicted_adjusted))
print("rmse", mean_squared_error(z_test, test_predict))